#### Various functions to extract or manipulate date data

In [ ]:
# Needed libraries
import numpy as np

Get date from file name and split

In [ ]:
# Get filedate string - takes file name
# Returns 1 value: string of date/data
def filedate(x):
    return(x[-14:-4])

# Split data into ymd - takes date in y_m_d format
# Returns 3 values: year, month, day
def splitter(date):
    y = int(date.split('-')[0])
    m = int(date.split('-')[1])
    d = int(date.split('-')[2])
    return y, m, d

Check for leap year

In [ ]:
# If year is a leap year return TRUE; else FALSE
def checkLeap(year):
   if((year % 400 == 0) or (year % 100 != 0) and (year % 4 == 0)):
       return True
   return False

def is_leap_year(year):
    if year % 100 == 0:
        return year % 400 == 0
    return year % 4 == 0

Get DOY in different formats

In [ ]:
# DOY - takes YMD; uses leap year check
# Returns 1 value: doy
def doy(Y,M,D):
    """ given year, month, day return day of year
        Astronomical Algorithms, Jean Meeus, 2d ed, 1998, chap 7 """
    if is_leap_year(Y):
        K = 1
    else:
        K = 2
    N = int((275 * M) / 9.0) - K * int((M + 9) / 12.0) + D - 30
    return N

# Cyclical DOY - takes doy and year; uses leap year check
# Returns 2 values: sin and cos doy
def cyclical_doy(doy, Y):
    if is_leap_year(Y):
        doy_sin = np.sin(2 * np.pi * doy/366.)
        doy_cos = np.cos(2 * np.pi * doy/366.)
        return doy_sin, doy_cos
    else:
        doy_sin = np.sin(2 * np.pi * doy/365.)
        doy_cos = np.cos(2 * np.pi * doy/365.)
        return doy_sin, doy_cos

# Cumulative DOY - takes image name; uses filedate, splitter, doy
# Returns 1-2 values: cumulative doy and optionally year
def cumulative_doy(image):
    begin_y = 2017
    leap_year = [1956,1960,1964,1968,1972,1976,1980,1984,1988,1992,1996,2000,
                 2004,2008,2012,2016,2020,2024,2028,2032,2036,2040,2044,2048]
    date = filedate(image)
    Y,M,D = splitter(date)
    doy = get_doy(Y,M,D)
    if Y == begin_y:
        return doy, Y
    else:
        year = begin_y
        count = 0
        while Y > year:
            if year in leap_year:
                count += 366
            else:
                count += 365
            year += 1
        cum_doy = doy + count
        return cum_doy #Y

Navigating multiple files/folders at once

In [ ]:
# Get all date info from file - takes file name
# Requires filedate, splitter, all doy variants
# Returns 5 values: year, doy: original, cumulative, sin and cos
def one_step(image):
  date = filedate(image)
  y,m,d = splitter(date)
  dofy = get_doy(y,m,d)
  doy_sin, doy_cos = cyclical_doy(dofy,y)
  cum_doy = cumulative_doy(image)
  return y, dofy, cum_doy, doy_sin, doy_cos

# Get dates from all files in a folder - takes folder name
# Requires fieldate, splitter, all doy variants
# Returns dictionary of values (same as one_step)
def date_data(folder):
    # Define a function to get file data, based on naming scheme
    # In this project [-14] will include YMD
    def filedate(x):
        return(x[-14:])

    # empty dictionary to be filled with date data as follows:
        # key: image name, values: Y, M, D, doy, doy_sin, doy_cos
    date_dict = {}
    import time_conversions as time
    for image in sorted(folder, key=filedate):
        if image.endswith('.tif'):
            dict_values = [] # create new list each image
            
        # files used have filenames 33UWP_(number)_YEAR-MONTH-DAY.tif
            dateTif = image.split('_')[2]
            date = dateTif.split('.')[0]
            y,m,d = splitter(date)
            doy = doy(y,m,d)  # doy from date
            doy_sin, doy_cos = cyclical_doy(doy, y)  # cyclical feature
            cum_doy = cumulative_doy(image)
            dict_values.extend(y,m,d,doy,doy_sin,doy_cos, cum_doy)
            date_dict[image] = dict_values
    return date_dict